## 기본설정 및 함수정의

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# tqdm의 pandas전용 메소드를 호출
tqdm.pandas()
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 목표표 위도, 경도
hwasung = [37.20387682941874, 127.0634311210021]
 
# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.progress_apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 격자

In [12]:
# GeoJSON 파일 불러오기
with open('SBJ_2411_001/1-14.화성시_격자.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
grid_map_df = grid_map_df[~grid_map_df['properties.gid'].str.startswith('다아')]
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 상세 도로망

In [14]:
# GeoJSON 파일 불러오기
with open('SBJ_2411_001/1-3.화성시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

#### 추정교통량

In [5]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

traffic_df = pd.read_csv('SBJ_2411_001/1-2.화성시_추정교통량.csv')
traffic_df['year'] = traffic_df['year'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df = traffic_df[traffic_df['timeslot'] == 'all'] # 일별 누적교통량 사용
traffic_df['geometry'] = traffic_df['link_id'].map(road_geo_dic)
traffic_df = gpd.GeoDataFrame(traffic_df, geometry='geometry')
traffic_df

C:\Users\user\AppData\Local\Temp\ipykernel_19112\579841776.py:4: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



,year,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,geometry
320898,2020,571351343,101,0.541,경부고속도로,경기도,화성시,동탄5동,all,81671.786596,59687.423761,4111.642485,17872.720350,"LINESTRING (127.09852 37.21898, 127.09805 37.2..."
320899,2020,571378519,101,1.830,경부고속도로,경기도,화성시,동탄5동,all,85023.990770,61021.431345,2834.578979,21167.980446,"LINESTRING (127.09622 37.21448, 127.09613 37.2..."
320900,2020,571378512,101,2.176,경부고속도로,경기도,화성시,동탄5동,all,86068.896137,62599.804550,3216.240412,20252.851176,"LINESTRING (127.09434 37.19789, 127.09438 37.1..."
320901,2020,571301261,101,0.074,경부고속도로,경기도,화성시,동탄6동,all,83481.346141,58070.191728,5756.302981,19654.851431,"LINESTRING (127.09655 37.18362, 127.09654 37.1..."
320902,2020,571301262,101,0.080,경부고속도로,경기도,화성시,동탄6동,all,87376.404828,60935.612353,5510.562526,20930.229949,"LINESTRING (127.09510 37.18416, 127.09526 37.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947383,2022,477352238,106,0.283,화성로,경기도,화성시,비봉면,all,19867.000000,16550.000000,104.000000,3213.000000,"LINESTRING (126.86165 37.23602, 126.86166 37.2..."
947384,2022,476340679,106,0.145,마도로,경기도,화성시,마도면,all,1989.000000,1299.000000,25.000000,665.000000,"LINESTRING (126.74813 37.18962, 126.74793 37.1..."
947385,2022,478252770,107,0.012,상신하길로126번길,경기도,화성시,향남읍,all,7857.000000,5760.000000,294.000000,1804.000000,"LINESTRING (126.90151 37.10408, 126.90149 37.1..."
947386,2022,477358945,106,0.024,화성로,경기도,화성시,비봉면,all,25772.000000,21176.000000,477.000000,4119.000000,"LINESTRING (126.85998 37.22982, 126.85999 37.2..."


In [16]:
# 맵 객체 생성 및 데이터 로드
road_estimating_map = KeplerGl(height=1000, width=1500)
road_estimating_map.add_data(data=traffic_df, name="추정교통량 데이터")
road_estimating_map.add_data(data=grid_map_df, name="격자 데이터")
# 맵 출력 및 상세설정
road_estimating_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\user\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\jupyter_client\session.py:719: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



KeplerGl(data={'추정교통량 데이터': {'index': [320898, 320899, 320900, 320901, 320902, 320903, 320904, 320905, 320906,…

#### 시설

##### 버스정류장 & 지하철역

In [9]:
pd.read_csv('SBJ_2411_001/1-5.화성시_버스정류장.csv')

,bis_id,stn_nm,lon,lat
0,GGB233001446,요당리,126.936550,37.068417
1,GGB233001790,신왕2리마을회관,126.949800,37.072233
2,GGB233001098,신왕2리.독줄입구,126.954083,37.077617
3,GGB233000022,독줄입구,126.953750,37.077883
4,GGB233001099,신왕3리.조리울,126.952267,37.080050
...,...,...,...,...
3004,GGB233002060,늘봄초교.파크푸르지오,127.111183,37.216367
3005,GGB233002182,파크푸르지오,127.112833,37.216650
3006,GGB233002187,파크푸르지오,127.112717,37.216750
3007,GGB233002169,LH1단지.한양수자인,127.114250,37.216767


In [10]:
pd.read_csv('SBJ_2411_001/1-6.화성시_지하철역.csv')

,stn_nm,stn_ln_info,lon,lat
0,병점역,경부선,127.033268,37.206821
1,어천역,수인선,126.908805,37.250076
2,야목역,수인선,126.884290,37.261011
3,서동탄,경부선,127.051672,37.195504
4,동탄,수도권 광역급행철도,127.095690,37.200340
5,서화성,서해선,126.787141,37.248709
6,화성시청,서해선,126.821003,37.189305
7,향남,서해선,126.913247,37.123283


##### 상권

In [8]:
pd.read_csv('SBJ_2411_001/1-4.화성시_상권정보.csv')

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,emd_cd,emd_nm,road_address,lon,lat,std_year
0,F,생활서비스,F14,자동차/이륜차,F14A01,자동차정비/카센타,NaN,NaN,4159036000,팔탄면,경기도 화성시 팔탄면 서해로987번길 7,126.891262,37.149846,2021
1,D,소매,D01,음/식료품소매,D01A01,식료품점,G47219,기타 식료품 소매업,4159034000,송산면,경기도 화성시 송산면 당성로 571,126.723828,37.208587,2021
2,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,화장품 및 방향제 소매업,4159058500,동탄1동,경기도 화성시 동탄공원로3길 18-23,127.062309,37.207623,2021
3,D,소매,D08,사무/문구/컴퓨터,D08A03,컴퓨터/사무용기기종합,G47311,"컴퓨터 및 주변장치, 소프트웨어 소매업",4159025900,향남읍,경기도 화성시 향남읍 평1길 16,126.909272,37.132694,2021
4,Q,음식,Q01,한식,Q01A02,갈비/삼겹살,I56111,한식 음식점업,4159025900,향남읍,경기도 화성시 향남읍 행정중앙1로 14,126.916244,37.125963,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130871,I2,음식,I201,한식,I20101,백반/한정식,I56111,한식 일반 음식점업,41590256,우정읍,경기도 화성시 우정읍 기아자동차로 95,126.788963,37.039253,2024
130872,G2,소매,G211,가구 소매,G21101,가구 소매업,G47520,가구 소매업,41590590,동탄5동,경기도 화성시 동탄기흥로520번가길 17-1,127.093276,37.208820,2024
130873,I2,음식,I210,기타 간이,I21003,피자,I56192,피자; 햄버거; 샌드위치 및 유사 음식점업,41590587,동탄3동,경기도 화성시 효행로 1260,127.060238,37.221088,2024
130874,I2,음식,I201,한식,I20101,백반/한정식,I56111,한식 일반 음식점업,41590630,동탄9동,경기도 화성시 동탄물류1로 13,127.126245,37.172230,2024
